In [91]:
# import dependencies
import pandas as pd
import numpy as np
import sys
from datetime import datetime
import getpass

# # For MySQL
# from sqlalchemy import create_engine
# import pandas as pd
# import pymysql

# For PostgreSQL
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

# # For SQL Server
# from sqlalchemy import create_engine
# import pandas as pd
# import pyodbc

In [92]:
import psycopg2

username = 'gmb'
password = getpass.getpass('Enter password:')
public_ip_address = '34.28.219.203'
db_name = 'gmb'

# Create a connection
conn = psycopg2.connect(
    dbname=db_name,
    user=username,
    password=password,
    host=public_ip_address
)

# Create a cursor
cur = conn.cursor()

Enter password:········


In [118]:
# Query to get all table names in the PostgreSQL database
query = """
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname != 'pg_catalog' AND 
      schemaname != 'information_schema';
"""

# Execute the query
cur.execute(query)

# Anonymization

In [119]:
query = """

SELECT distinct table_name
        FROM information_schema.columns
WHERE column_name = 'first_name' 
   OR column_name = 'last_name'
   OR column_name = 'phone'
   OR column_name = 'email'
   OR column_name = 'zipcode'
   OR column_name = 'password'
 
"""
sensitive_tables = pd.read_sql(query, conn)
sensitive_tables

,table_name
0,billing_paymentcustomer
1,challenge_challengeinvite
2,group_groupinvite
3,matching_matchingsettings
4,member_user
5,social_auth_code


In [120]:
# billing_paymentcustomer
query_billing = "select * from billing_paymentcustomer"
biling_paymentcustomer_df = pd.read_sql(query_billing, conn)

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [121]:
biling_paymentcustomer_anonydf = billing_paymentcustomer_df.copy()

biling_paymentcustomer_anonydf['first_name'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'FirstUser{}'.format(x))

biling_paymentcustomer_anonydf['last_name'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'LastUser{}'.format(x))

biling_paymentcustomer_anonydf['email'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'user{}@example.com'.format(x))

if 'phone' in biling_paymentcustomer_anonydf.columns:
    biling_paymentcustomer_anonydf['phone'] = biling_paymentcustomer_anonydf['id']\
    .apply(lambda x: '123456{}'.format(x))
    
if 'created_from_ip' in biling_paymentcustomer_anonydf.columns:
    biling_paymentcustomer_anonydf['created_from_ip'] = "192.168.1.1" 
    


**About anonymizing IP address:**
"192.168.1.1" is a commonly used default IP address for many routers and home networking equipment. It falls within the range of IP addresses (192.168.0.0 to 192.168.255.255) reserved for private networks, meaning these addresses are not routable on the public internet and are often used for local network devices.

In [123]:
db_url = f"postgresql://{username}:{password}@{public_ip_address}/{db_name}"

# SQLAlchemy engine
engine = create_engine(db_url)

when using the `to_sql()` method in pandas, you need to provide an SQLAlchemy engine, not a psycopg2 connection. This is because pandas uses SQLAlchemy to handle SQL dialects.

In [124]:
biling_paymentcustomer_anonydf.to_sql('billing_paymentcustomer_anonymized', engine, if_exists='replace', index=False)


590